In [ ]:
import pandas as pd

df = pd.read_html('..spac_list.html')[0]
df.to_csv('../data/spac_list.csv')

## yfinance 로 합병 전 SPAC 의 주가 데이터를 불러올 수 있는지 본다.

합병 전 ticker 는 보통 spac 스폰서 회사에서 계속 반복해서 티커명을 사용하기 때문에 데이터를 불러올 수 없다. 합병 후 기업의 티커명으로 당시 주가를 찾아야 한다.

yfinance 기준으로 합병 이전 데이터가 있는 회사도 있고 없는 회사도 있다.

예시로 `MGY` 종목은 합병 이전 주가가 차트에 반영되어있고, `LAZY` 는 그렇지 않다.

In [15]:
feasible_spac = 'MGY'
unfeasible_spac = 'LAZY'

import yfinance as yf

mgy = yf.Ticker(feasible_spac)
lazy = yf.Ticker(unfeasible_spac)


mgy_hist = mgy.history(start="2017-01-01", end="2018-07-31")
lazy_hist = lazy.history(start="2017-01-01", end="2018-03-15") # no data

mgy_hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-06-29,10.00,10.00,10.000,10.00,1100,0,0
2017-06-30,10.25,10.25,10.050,10.06,3300,0,0
2017-07-03,10.06,10.06,10.060,10.06,0,0,0
2017-07-05,10.06,10.06,10.060,10.06,0,0,0
2017-07-06,10.06,10.06,10.060,10.06,0,0,0
...,...,...,...,...,...,...,...
2018-07-24,12.64,12.85,12.330,12.78,1967700,0,0
2018-07-25,12.99,13.00,12.680,12.76,1386900,0,0
2018-07-26,12.71,12.87,12.355,12.39,907800,0,0


따라서 로직은 다음과 같이 구현되어야 한다.

1. `spac_list.csv` 로 부터 "티커", "상장일", "합병일" 데이터를 가져온다.
2. yfinance 를 이용해서 종목별로 상장일부터 합병일 까지의 시세 데이터를 가져온다.
3. 해당 일의 시세 데이터를 가져오지 못한 종목을 제외한다.
4. 시세 데이터를 DB 에 저장한다.

In [4]:
import json
import pymysql

db_config = json.loads(open('../env/db_config.json', 'r').read())
connection = pymysql.connect(host=db_config['host'],
                             port=db_config['port'],
                             db=db_config['db'],
                             user=db_config['user'],
                             passwd=db_config['passwd'],
                             autocommit=True,
                             charset='utf8'
                             )


In [11]:
import yfinance
import pandas as pd

def insert_prices (ticker, ohlc_prices, conn):
    for r in ohlc_prices.itertuples():
        sql = f"REPLACE INTO prices values ('{ticker}', "\
          f"'{r.Index}', '{r.Open}', '{r.High}', "\
          f"'{r.Low}', '{r.Close}', '{r.Volume}')"

        with conn.cursor() as curs:
            curs.execute(sql)


def fetch_price(ticker, start, end):
    stock = yfinance.Ticker(ticker)
    return stock.history(start=start, end=end)


def get_spacs(from_ipo_date, to_ipo_date):
    df = pd.read_csv('../data/spac_list.csv')

    df = df[df['SPAC IPO Date'] >= from_ipo_date]
    if(to_ipo_date):
        df = df[df['SPAC IPO Date'] >= to_ipo_date]
    return df


def update_spac_prices(from_ipo_date):
    spacs = get_spacs(from_ipo_date)

    for r in spacs.itertuples():
        print(f"Fetching {r[3]}")

        ohlc_prices = fetch_price(r[2], r[15], r[6])
        insert_prices(r[2], ohlc_prices, connection)

# update_spac_prices('2017-01-01')

## SPAC 상장일 까지의 평균 기간

연도별 histogram 생성

In [14]:
spac_2017 = get_spacs('2017-01-01', '2017-12-31')
spac_2018 = get_spacs('2018-01-01', '2018-12-31')
spac_2019 = get_spacs('2019-01-01', '2019-12-31')
spac_2020 = get_spacs('2020-01-01', '2020-12-31')

def append_ipo_to_merger(spacs):
    print(spacs['Merger Completion Date (Announced)'] - \
          spacs['SPAC IPO Date'])

append_ipo_to_merger(spac_2017)

TypeError: unsupported operand type(s) for -: 'str' and 'str'